# Отток клиентов банка

**Описание проекта**

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением F1-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте F1-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте AUC-ROC, сравнивайте её значение с F1-мерой.

План работы:

1. Загрузите и подготовьте данные.
2. Исследуйте баланс классов, обучите модель без учёта дисбаланса.
3. Улучшите качество модели, учитывая дисбаланс классов. Обучите разные модели и найдите лучшую.
4. Проведите финальное тестирование.

**Описание данных**

Признаки

1. `RowNumber` — индекс строки в данных
2. `CustomerId` — уникальный идентификатор клиента
3. `Surname` — фамилия
4. `CreditScore` — кредитный рейтинг
5. `Geography` — страна проживания
6. `Gender` — пол
7. `Age` — возраст
8. `Tenure` — сколько лет человек является клиентом банка
9. `Balance` — баланс на счёте
10. `NumOfProducts` — количество продуктов банка, используемых клиентом
11. `HasCrCard` — наличие кредитной карты
12. `IsActiveMember` — активность клиента
13. `EstimatedSalary` — предполагаемая зарплата

Целевой признак

1. `Exited` — факт ухода клиента

In [1]:
import pandas as pd
import optuna
import plotly.graph_objects as go

from caseconverter import snakecase
from ydata_profiling import ProfileReport
from IPython.display import display
from tqdm import tqdm

from fast_ml import eda
from fast_ml.model_development import train_valid_test_split

from sklearn.utils import resample

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier

from sklearn.metrics import (
    accuracy_score, f1_score, auc, roc_curve, roc_auc_score
)

In [2]:
FIG_WIDTH = 10 * 100
FIG_HEIGHT = 5 * 100
RANDOM_SEED = 42

In [3]:
try:
    raw_users = pd.read_csv('Churn.csv')
except:
    raw_users = pd.read_csv('/datasets/Churn.csv')

## Исследовательский анализ данных

Изучим основные зависимости в данных перед тем, как мы будем использовать их в алгоритмах машинного обучения.

Таблица-резюме:

In [4]:
display(eda.df_info(raw_users))

,data_type,data_type_grp,num_unique_values,sample_unique_values,num_missing,perc_missing
RowNumber,int64,Numerical,10000,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]",0,0.0
CustomerId,int64,Numerical,10000,"[15634602, 15647311, 15619304, 15701354, 15737...",0,0.0
Surname,object,Categorical,2932,"[Hargrave, Hill, Onio, Boni, Mitchell, Chu, Ba...",0,0.0
CreditScore,int64,Numerical,460,"[619, 608, 502, 699, 850, 645, 822, 376, 501, ...",0,0.0
Geography,object,Categorical,3,"[France, Spain, Germany]",0,0.0
Gender,object,Categorical,2,"[Female, Male]",0,0.0
Age,int64,Numerical,70,"[42, 41, 39, 43, 44, 50, 29, 27, 31, 24]",0,0.0
Tenure,float64,Numerical,11,"[2.0, 1.0, 8.0, 7.0, 4.0, 6.0, 3.0, 10.0, 5.0,...",909,9.09
Balance,float64,Numerical,6382,"[0.0, 83807.86, 159660.8, 125510.82, 113755.78...",0,0.0
NumOfProducts,int64,Numerical,4,"[1, 3, 2, 4]",0,0.0


Числовые распределения:

In [5]:
display(round(raw_users.describe().T, 2))

,count,mean,std,min,25%,50%,75%,max
RowNumber,10000.0,5000.50,2886.90,1.00,2500.75,5000.50,7500.25,10000.00
CustomerId,10000.0,15690940.57,71936.19,15565701.00,15628528.25,15690738.00,15753233.75,15815690.00
CreditScore,10000.0,650.53,96.65,350.00,584.00,652.00,718.00,850.00
Age,10000.0,38.92,10.49,18.00,32.00,37.00,44.00,92.00
Tenure,9091.0,5.00,2.89,0.00,2.00,5.00,7.00,10.00
Balance,10000.0,76485.89,62397.41,0.00,0.00,97198.54,127644.24,250898.09
NumOfProducts,10000.0,1.53,0.58,1.00,1.00,1.00,2.00,4.00
HasCrCard,10000.0,0.71,0.46,0.00,0.00,1.00,1.00,1.00
IsActiveMember,10000.0,0.52,0.50,0.00,0.00,1.00,1.00,1.00
EstimatedSalary,10000.0,100090.24,57510.49,11.58,51002.11,100193.92,149388.25,199992.48


И детальный отчет:

In [6]:
ProfileReport(raw_users).to_widgets()

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

Ключевые наблюдения из предварительного анализа набора данных:

1. **Демографические данные клиентов**: Набор данных содержит информацию о 10 000 клиентах с различными характеристиками. Средний возраст клиентов составляет 39 лет, с разбросом от 18 до 92 лет. В наборе данных представлены клиенты из трех стран: Франции, Испании и Германии. Кроме того, присутствует почти равномерное распределение мужчин и женщин.

2. **Финансовые показатели**: Набор данных включает финансовые метрики, такие как кредитные баллы, балансы счетов, количество продуктов и зарплаты. Средний кредитный балл составляет 651, что указывает на относительно хорошую кредитоспособность. Средний баланс счета составляет 76К долларов, с широким диапазоном значений. Клиенты имеют в среднем 1,53 продукта в банке, а зарплаты варьируются от 11 до 200К долларов.

3. **Отток и вовлеченность**: Около 20% клиентов в наборе данных покинули или прекратили использовать услуги банка. Приблизительно 52% клиентов являются активными участниками, что указывает на умеренный уровень вовлеченности. Средняя продолжительность работы с клиентом составляет 5 лет, хотя в этой колонке есть пропущенных значений для некоторых записей.

4. **Целевой признак**: У нас сть дисбаланс классов в целевом признаке `exited` - около 8К записей оставшихся пользователей и только 2К записей пользователей, которые перестали быть пользователями нашего банка. Это надо будет выровнять, чтобы избежать model bias.

# Подготовка данных для ML

Выполним эти преобразования на полном датасете:

1. Уберем колонки, которые не нужны для моделей: `RowNumber`, `CustomerId`, `Surname`.
2. Переведем колонку `gender` в бинарную: `1` - для мужчин и `0` - для женщин.
3. Названия колонок приведем `snake_case` регистру.
4. Проведем `upsampling`.

И остальные после разделения на выборки, чтобы избежать data leakage:

1. Заполним пропуски в `tenure`.
2. Проведем стандартизацию численных признаков.
3. Проведем кодирование категориальных признаков.

In [7]:
df_no_sampling = (
    raw_users
    .copy()
    .drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)
    .replace({'Male': 1, 'Female': 0})
    .rename(columns=lambda column: snakecase(column))
    .assign(geography=lambda df: df.geography.str.lower())
)

display(df_no_sampling.head())


,credit_score,geography,gender,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited
0,619,france,0,42,2.0,0.00,1,1,1,101348.88,1
1,608,spain,0,41,1.0,83807.86,1,0,1,112542.58,0
2,502,france,0,42,8.0,159660.80,3,1,0,113931.57,1
3,699,france,0,39,1.0,0.00,2,0,0,93826.63,0
4,850,spain,0,43,2.0,125510.82,1,1,1,79084.10,0


Выглядит адекватно. Сделаем `upsampling`.

In [8]:
# Perform the split between majority and minority, upsample minority, and concatenate results
df_upsampled = (
    pd.concat([
        df_no_sampling[df_no_sampling.exited==0],
        resample(
            df_no_sampling[df_no_sampling.exited==1],
            replace=True,
            n_samples=len(df_no_sampling[df_no_sampling.exited==0]),
            random_state=RANDOM_SEED
        )
    ])
)

Проверим, что получилось.

In [9]:
def print_class_distribution(df, column, label):
    print(
        f"{'-' * 40}\n{label} distribution, %:\n{df[column].value_counts(normalize=True) * 100}\n{'-' * 40}"
    )

print_class_distribution(df_no_sampling, 'exited', 'Exited without resampling')
print_class_distribution(df_upsampled, 'exited', 'Exited with upsampling')

----------------------------------------
Exited without resampling distribution, %:
0    79.63
1    20.37
Name: exited, dtype: float64
----------------------------------------
----------------------------------------
Exited with upsampling distribution, %:
0    50.0
1    50.0
Name: exited, dtype: float64
----------------------------------------


В `df_upsampled` классы теперь сбалансированы.

Разделим датасеты на выборки и выполним остальные преобразовния.

In [10]:
# Define variables
data_splits = {}
split_keys = ['ftr_train', 'tgt_train', 'ftr_valid', 'tgt_valid', 'ftr_test', 'tgt_test']
num_cols = ['credit_score', 'age', 'tenure', 'balance', 'num_of_products', 'estimated_salary']
cat_cols = ['geography']
bin_cols = ['has_cr_card', 'is_active_member', 'gender']

# Define the preprocessing pipelines
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
])

cat_pipeline = Pipeline([
    ('encoder', OneHotEncoder(handle_unknown='ignore')),
])

preprocessor = ColumnTransformer([
    ('num', num_pipeline, num_cols),
    ('cat', cat_pipeline, cat_cols),
])


for dataset, label in zip([df_no_sampling, df_upsampled], ['no_sampling', 'upsampled']):
    splits = train_valid_test_split(
        dataset, 'exited', train_size=0.7, valid_size=0.15, test_size=0.15, random_state=RANDOM_SEED
    )
    
    # Use the split keys and the splits to create a dictionary for each dataset
    data_splits[label] = dict(zip(split_keys, splits))
    
    # Fit the preprocessor on the training data only
    preprocessor.fit(data_splits[label]['ftr_train'])

    # Transform and reformat each subset of the data
    for subset in split_keys:
        # Only transform feature subsets
        if 'ftr' in subset:
            data_transformed = preprocessor.transform(data_splits[label][subset])
            
            columns_transformed = num_cols + list(
                preprocessor.named_transformers_['cat']['encoder'].get_feature_names_out(cat_cols)
            )
            df_transformed = pd.DataFrame(
                data_transformed, columns=columns_transformed
            )
            df_transformed = pd.concat(
                [df_transformed, data_splits[label][subset][bin_cols].reset_index(drop=True)], axis=1
            )

            data_splits[label][subset] = df_transformed

Проверим, что получилось на одном примере.

In [11]:
data_splits['no_sampling']['ftr_valid'].head()

,credit_score,age,tenure,balance,num_of_products,estimated_salary,geography_france,geography_germany,geography_spain,has_cr_card,is_active_member,gender
0,-0.012043,-0.371195,1.098210,0.538578,-0.903352,0.465751,0.0,1.0,0.0,1,0,1
1,-0.012043,-0.466380,-0.355822,0.242578,-0.903352,0.401147,1.0,0.0,0.0,1,1,1
2,-1.643626,-0.085639,0.734702,1.576231,-0.903352,-1.053519,0.0,1.0,0.0,1,0,0
3,0.829729,1.437322,-1.809855,0.925391,-0.903352,-1.516265,0.0,0.0,1.0,0,1,1
4,0.091879,-0.371195,0.007686,0.832067,-0.903352,0.725006,0.0,0.0,1.0,1,0,0


В этом скрипте мы успешно обработали два набора данных, `df_no_sampling` и `df_upsampled`. На первом этапе были определены переменные и настроен предпроцессор предварительной обработки для числовых и категориальных столбцов, бинарные столбцы остались без изменений.

Основной цикл скрипта выполняет три основные задачи для каждого набора данных:

1. Разделение данных: Используется функция `train_valid_test_split`, чтобы разделить каждый набор данных на обучающие, валидационные и тестовые подмножества.

2. Подгонка препроцессора: Препроцессор обучается на обучающих данных. Это включает в себя изучение параметров для `imputation` наиболее часто встречающихся значений и стандартного масштабирования.

3. Трансформация данных: Используя подготовленный препроцессор, трансформируются признаки для каждого набора и типа колонок.

После выполнения этого скрипта у нас есть все подмножества данных, готовые для дальнейшего обучения модели и оценки.

# ML модели

Перейдем к обучению моделей. Воспользуемся библиотекой `optuna` чтобы найти оптимальную модель.

Для начала - сделаем функцию, которая будет выбирать для нас лучшую модель.

In [12]:
def get_study(dataset: str, n_trials: int):
    """
    Trains and optimizes Logistic Regression, Decision Tree, and Random Forest models using Optuna.

    Args:
        - dataset (str): The label of the dataset to use from the 
          `data_splits` dictionary('no_sampling' or 'upsampled').
        - n_trials (int): The number of trials for Optuna optimization.

    Returns:
        - An Optuna study object, containing the optimal model and its parameters.
    """
    
    def objective(trial):
        classifier_name = trial.suggest_categorical(
            'classifier', ['LogisticRegression', 'DecisionTree', 'RandomForest']
        )
        
        # Suggest parameters based on the classifier
        if classifier_name == 'LogisticRegression':
            lr_c = trial.suggest_float('lr_c', 1e-6, 1e6, log=True)
            classifier_obj = LogisticRegression(
                C=lr_c, random_state=RANDOM_SEED
            )
        elif classifier_name == 'DecisionTree':
            dt_max_depth = trial.suggest_int('dt_max_depth', 1, 50)
            classifier_obj = DecisionTreeClassifier(
                max_depth=dt_max_depth, random_state=RANDOM_SEED
            )
        elif classifier_name == 'RandomForest':
            rf_max_depth = trial.suggest_int('rf_max_depth', 1, 100)
            rf_n_estimators = trial.suggest_int('rf_n_estimators', 100, 1000)
            classifier_obj = RandomForestClassifier(
                max_depth=rf_max_depth,
                n_estimators=rf_n_estimators,
                random_state=RANDOM_SEED
            )
            
        classifier_obj.fit(data_splits[dataset]['ftr_train'], data_splits[dataset]['tgt_train'])
        
        prediction = classifier_obj.predict(data_splits[dataset]['ftr_valid'])
        
        f1 = f1_score(data_splits[dataset]['tgt_valid'], prediction)
        
        return round(f1, 3)

    optuna.logging.set_verbosity(optuna.logging.WARNING)
    study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=RANDOM_SEED))
    study.optimize(objective, n_trials=n_trials)
    
    return study

Теперь функцию, которая посчитает метрики для модели.

In [13]:
def evaluate_best_model(dataset: str, study: optuna.study.Study) -> dict[str, float]:
    """
    Given a dataset label (key in the data_splits dictionary) and an Optuna study object,
    this function retrieves the best model parameters from the study, instantiates a new
    model with these parameters, makes predictions on the validation and test subsets,
    and calculates and returns the F1 score and ROC AUC score for each subset.

    Args:
        - dataset (str): Label of the dataset to use (should be a key in the data_splits dictionary).
        - study (optuna.study.Study): Optuna study object containing the results of model optimization.

    Returns:
        - dict: A dictionary with the F1 score and ROC AUC score for the validation and test subsets.
    """
    
    best_params = study.best_params

    if best_params['classifier'] == 'LogisticRegression':
        best_model = LogisticRegression(
            C=best_params['lr_c'], random_state=RANDOM_SEED
        )
    elif best_params['classifier'] == 'DecisionTree':
        best_model = DecisionTreeClassifier(
            max_depth=best_params['dt_max_depth'], random_state=RANDOM_SEED
        )
    elif best_params['classifier'] == 'RandomForest':
        best_model = RandomForestClassifier(
            max_depth=best_params['rf_max_depth'],
            n_estimators=best_params['rf_n_estimators'],
            random_state=RANDOM_SEED
        )

    best_model.fit(data_splits[dataset]['ftr_train'], data_splits[dataset]['tgt_train'])

    pred_valid = best_model.predict(data_splits[dataset]['ftr_valid'])
    pred_test = best_model.predict(data_splits[dataset]['ftr_test'])

    f1_valid = f1_score(data_splits[dataset]['tgt_valid'], pred_valid)
    roc_auc_valid = roc_auc_score(data_splits[dataset]['tgt_valid'], pred_valid)

    f1_test = f1_score(data_splits[dataset]['tgt_test'], pred_test)
    roc_auc_test = roc_auc_score(data_splits[dataset]['tgt_test'], pred_test)

    return {
        'valid_f1': f1_valid,
        'valid_roc_auc': roc_auc_valid,
        'test_f1': f1_test,
        'test_roc_auc': roc_auc_test
    }, best_model


Выведет результаты.

In [ ]:
def print_best_params_and_metrics(best_params: dict, metrics: dict):
    """
    Prints the best parameters of the model and the performance metrics.

    Args:
        - best_params (dict): Dictionary of the best parameters from the model.
        - metrics (dict): Dictionary of performance metrics.

    Returns:
        - None
    """
    print('The best model parameters are:)
    for param, value in best_params.items():
        print(f"{param}: {value}")

    print('\nThe performance metrics of the model are:')
    for metric, value in metrics.items():
        print(f"{metric}: {value:.4f}")

И нарисует график `ROC AUC`.

In [14]:
def plot_roc_auc(dataset: str, best_model: any, title: str) -> go.Figure:
    """
    Plots ROC curves for the validation and test sets using Plotly.

    Args:
        - dataset (str): Label of the dataset to use (should be a key in the data_splits dictionary).
        - best_model (Any): Trained model object.
        - title (str): Title of the plot.

    Returns:
        - fig (go.Figure): Plotly figure with ROC curves.
    """

    probas_valid = best_model.predict_proba(data_splits[dataset]['ftr_valid'])
    fpr_valid, tpr_valid, _ = roc_curve(data_splits[dataset]['tgt_valid'], probas_valid[:, 1])
    roc_auc_valid = auc(fpr_valid, tpr_valid)
    
    probas_test = best_model.predict_proba(data_splits[dataset]['ftr_test'])
    fpr_test, tpr_test, _ = roc_curve(data_splits[dataset]['tgt_test'], probas_test[:, 1])
    roc_auc_test = auc(fpr_test, tpr_test)
    
    fig = go.Figure()

    fig.add_trace(
        go.Scatter(
            x=fpr_valid, y=tpr_valid,
            mode='lines',
            name=f'Validation set ROC (AUC = {roc_auc_valid:.2f})'
        )
    )
    fig.add_trace(
        go.Scatter(
            x=fpr_test, y=tpr_test,
            mode='lines',
            name=f'Test set ROC (AUC = {roc_auc_test:.2f})'
        )
    )
    fig.add_trace(
        go.Scatter(
            x=[0, 1], y=[0, 1],
            mode='lines',
            name='No skill',
            line=dict(dash='dash', color='black')
        )
    )

    fig.update_layout(
        xaxis_title='False positive rate',
        yaxis_title='True positive rate',
        width=FIG_WIDTH, height=FIG_HEIGHT * 1.5,
        title=title,
        template='plotly_white'
    )

    return fig


## Датасет без sampling

Возьмем датасет как есть и посмотрим на качество моделей.

In [15]:
study_no_sampling = get_study('no_sampling', n_trials=10)

metrics, best_model = evaluate_best_model('no_sampling', study_no_sampling)

Визуализируем результаты.

In [23]:
print_best_params_and_metrics(study_no_sampling.best_params, metrics)

fig = optuna.visualization.plot_optimization_history(study_no_sampling)
fig.update_layout(template='plotly_white', width=FIG_WIDTH, height=FIG_HEIGHT)
fig.show()

fig = optuna.visualization.plot_slice(study_no_sampling)
fig.update_layout(template='plotly_white', width=FIG_WIDTH, height=FIG_HEIGHT)
fig.update_xaxes(tickangle=-90)
fig.show()

fig = plot_roc_auc('no_sampling', best_model, 'ROC for the best model, no sampling')
fig.show()

The best model parameters are:
classifier: RandomForest
rf_max_depth: 18
rf_n_estimators: 158

The performance metrics of the model are:
valid_f1: 0.5809
valid_roc_auc: 0.7158
test_f1: 0.5984
test_roc_auc: 0.7279


## Датасет с upsampling

Возьмем датасет c `upsampling` и посмотрим на качество моделей.

In [25]:
study_upsampled = get_study('upsampled', n_trials=10)

metrics, best_model = evaluate_best_model('upsampled', study_upsampled)

Визуализируем результаты.

In [26]:
print_best_params_and_metrics(study_upsampled.best_params, metrics)

fig = optuna.visualization.plot_optimization_history(study_upsampled)
fig.update_layout(template='plotly_white', width=FIG_WIDTH, height=FIG_HEIGHT)
fig.show()

fig = optuna.visualization.plot_slice(study_upsampled)
fig.update_layout(template='plotly_white', width=FIG_WIDTH, height=FIG_HEIGHT)
fig.update_xaxes(tickangle=-90)
fig.show()

fig = plot_roc_auc('upsampled', best_model, 'ROC for the best model, upsampled')
fig.show()

The best model parameters are:
classifier: RandomForest
rf_max_depth: 18
rf_n_estimators: 158

The performance metrics of the model are:
valid_f1: 0.9445
valid_roc_auc: 0.9434
test_f1: 0.9452
test_roc_auc: 0.9435


# Выводы

На основе результатов экспериментов мы наблюдаем значительные различия в метриках модели при использовании `upsamlping` по сравнению с оригинальным датасетом.

Лучшей моделью в обоих случаях оказался классификатор `RandomForest` с `max_depth = 18` и `n_estimators = 158`. Однако эффективность этой модели на двух наборах данных довольно различна.

Для набора данных без `upsampling`:

1. Модель достигает значения F1-меры примерно 0,58 на валидационном наборе и 0,60 на тестовом наборе.
2. Площадь под кривой (ROC-AUC) составляет около 0,72 для обоих наборов данных.

В случае с `upsampling`:

1. Значение F1-меры модели значительно улучшается, достигая около 0,94 как для валидационного, так и для тестового наборов данных.
2. ROC-AUC также значительно повышается, достигая приблизительно 0,94 для обоих наборов.

Метод `upsampling` эффективно справляется с дисбалансом классов в наборе данных, что приводит к улучшению качества предсказаний миноритарного класса. Поэтому, в этом конкретном случае, применение `upsampling` оказалось подходящим решением.

Важно отметить, что выбор `upsampling`, модели и ее гиперпараметров должен производиться тщательно, с учетом контекста проблемы и специфических характеристик данных. Кроме того, очень важно проверить, что модель не переобучается на данных с `upsampling`, и производительность на новых данных сохраняется.

Наконец, стоит рассмотреть другие метрики и деловые цели, помимо F1 и ROC-AUC, чтобы получить всестороннее понимание качества модели.
